# Módulos útiles para ficheros grib

Para abrir el notebook en Colab pulsa el botón:

<a href="https://colab.research.google.com/github/jlcasador/curso_formatos_meteo/blob/main/Modulos_para_grib.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[GRIB](https://confluence.ecmwf.int/display/CKB/What+are+GRIB+files+and+how+can+I+read+them) es un acrónimo cuyas siglas corresponden a "General Regularly distributed Information in Binary form".  
Es un [formato estándar de la Organización Meteorológica Mundial (OMM)](https://library.wmo.int/index.php?lvl=notice_display&id=10684) para archivar e intercambiar datos meteorológicos.

Esta foto puede representar la jerarquía de paquetes que vamos a usar en este curso:

![jerarquía de módulos](matroshka_grib_60.jpg)

### Instalación

#### Instalación de eccodes utilizando CMake

Para instalar eccodes podemos seguir las [instrucciones de instalación de la web del ECMWF](https://confluence.ecmwf.int/display/ECC/ecCodes+installation), como sigue: 

In [ ]:
!wget https://confluence.ecmwf.int/download/attachments/45757960/eccodes-2.30.0-Source.tar.gz?api=v2 -O eccodes-2.30.0-Source.tar.gz
!tar -xzf  /content/eccodes-2.30.0-Source.tar.gz
!mkdir build ; cd build
%cd build
!cmake -DCMAKE_INSTALL_PREFIX=eccodes ../eccodes-2.30.0-Source
!make -j 4
!make install
%cd /content

In [ ]:
# Para poder encontrar las grib tools
import os
PATH = os.getenv('PATH')
os.environ['PATH'] = ':'.join([PATH, '/content/build/eccodes/bin'])

In [ ]:
!codes_info

#### Instalación de eccodes con Conda

Otra alternativa es instalar conda, y después instalar eccodes desde conda:

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

In [ ]:
!conda install -q eccodes -c conda-forge

In [ ]:
!codes_info

#### Instalación de otros paquetes

(Notar que xarray está instalado por defecto en este entorno)

In [ ]:
!conda install -c conda-forge cfgrib

In [ ]:
!conda install metview-batch -c conda-forge
#!conda install metview -c conda-forge # si se quiere instalar metview completo (con GUI)

In [ ]:
!conda install metview-python -c conda-forge

In [ ]:
!conda install -c conda-forge cartopy

In [ ]:
!pip install climetlab

#### Ficheros de ejemplo

In [ ]:
!test -f ecmwf_surface.grib | wget https://owncloud.aemet.es/index.php/s/tpl9SuZ16HeByJA/download -O ecmwf_surface.grib
!test -f air_quality.grb2 | wget https://owncloud.aemet.es/index.php/s/XsRZh7OezHwcARx/download -O air_quality.grb2

### Estructura de un fichero grib. Grib_tools

- Unas palabras sobre gribs
- grib1 y grib2
- conversiones grib1 a grib2, a netcdf

https://library.wmo.int/index.php?lvl=notice_display&id=10684

Las [grib keys](https://confluence.ecmwf.int/display/ECC/GRIB+Keys) son metadatos de cada mensaje grib. Nos informan sobre características tales como.  
Algunas keys importantes son:
- centre: centro de origen del grib (notar que cada centro tendrá su tabla de definiciones)
- edition: indica si el fichero grib es de tipo 1 o 2
- gridType: tipo de grid (por ejemplo, regular_ll)
- table2Version: tabla de parámetros a la que pertenece el parámetro
- indicatorOfParameter: parámetro (para grib1)
- level: nivel
- typeOfLevel: tipo de nivel (superficie, nivel de presión ...)
- paramId: key computada que el ECMWF utiliza para facilitar la identificación del parámetro 
- dataDate: fecha de ejecución del modelo
- dataTime: hora de ejecución del modelo ("pasada")
- step: alcance (normalmente en horas)
- shortName: nombre corto del parámetro (depende de la tabla de definiciones)
- name: nombre largo del parámetro (depende de la tabla de definiciones)

Otras keys dan información sobre la distribución espacial de los puntos:
- longitudeOfFirstGridPointInDegrees
- latitudeOfFirstGridPointInDegrees
- longitudeOfLastGridPointInDegrees
- latitudeOfLastGridPointInDegrees
- iDirectionIncrementInDegrees
- jDirectionIncrementInDegrees
- Ni
- Nj

Las [grib tools](https://confluence.ecmwf.int/display/ECC/GRIB+tools) son un conjunto de comandos que permiten 

grib_ls muestra el contenido de algunas keys de un fichero grib. Da una línea por cada mensaje grib:

In [ ]:
!grib_ls ecmwf_surface.grib

In [ ]:
# Cuidado con mayúsculas y espacios!
!grib_ls -p paramId,shortName,dataDate,dataTime,step ecmwf_surface.grib

grib_dump nos da el valor de todas las keys codificadas del grib:

In [ ]:
!grib_dump ecmwf_surface.grib | head -n 120

Para aclarar qué código corresponden a cada parámetro podemos consultar las [tablas de parámetros](https://codes.ecmwf.int/grib/param-db/). Estas tablas son válidas para gribs procedentes del ECMWF.

### Módulo eccodes

https://confluence.ecmwf.int/display/ECC/ecCodes+Home

In [ ]:
import eccodes as ec

In [ ]:
f = open("ecmwf_surface.grib", "rb")

In [ ]:
gid = ec.codes_new_from_file(f, ec.CODES_PRODUCT_GRIB)

In [ ]:
ec.codes_get(gid, "paramId")

In [ ]:
ec.codes_release(gid)

In [ ]:
f.close()

Hay una [documentación en línea](https://sites.ecmwf.int/docs/eccodes/namespaceec_codes.html) sobre las distintas funciones de eccodes.

La API original está escrita en C, y permite manejar la memoria directamente leyendo cada uno de los mensajes del fichero grib:

![flujo basico C](flujo_basico_C.png)

La interfaz de python nos oculta la mayor parte de la gestión de memoria (pero cuidado con codes_release!), evitándonos este trabajo extra:

![flujo_basico_python](flujo_basico_python.png)

Algo un poco más complicado:

In [ ]:
def nearest_value(filename, input_paramid, latitude, longitude):
    
    f = open(filename, "rb")
    values = {}
    
    while True:
        gid = ec.codes_new_from_file(f, ec.CODES_PRODUCT_GRIB)
        if gid is None: break
        
        paramid = ec.codes_get(gid, "paramId")
        step = ec.codes_get(gid, "step")
        if paramid==input_paramid:
            values[step] = ec.codes_grib_find_nearest(gid, latitude, longitude)[0].value
        
        ec.codes_release(gid)

    f.close()
    
    return values

In [ ]:
nearest_value("ecmwf_surface.grib", 167, 40, -5)

### Módulo xarray

In [ ]:
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs

In [ ]:
ds = xr.load_dataset("ecmwf_surface.grib", engine="cfgrib")

Notar que xarray (con cfgrib) solo puede leer correctamente [gribs homogéneos](https://github.com/ecmwf/cfgrib/blob/master/README.rst)

Seleccionamos el campo de la temperatura a 2m, para el alcance h+12:

In [ ]:
t_h12 = ds.sel(step = timedelta(hours=12)).t2m - 273.15

In [ ]:
t_h12.plot()

O, si usamos cartopy para mostrar el mapa en proyección Mercator, por ejemplo, e incluimos la línea de costa:

In [ ]:
plt.figure(figsize=(8,4.5))
ax = plt.axes(projection=ccrs.Mercator())
ax.coastlines()
t_h12.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels= np.arange(-20,25,1))
plt.show()

### Módulo metview

[Metview](https://confluence.ecmwf.int/display/METV/Metview) ([documentación aquí](https://metview.readthedocs.io/en/latest/)) es una aplicación desarrollada por el ECMWF que proporciona un entorno completo de trabajo para meteorólogos. Da herramientas para el acceso, procesamiento y visualización de datos meteorológicos (ver la [galería](https://confluence.ecmwf.int/display/METV/Gallery)).

Nosotros la utilizaremos aquí solamente para ilustrar el [recorte e interpolación de campos](https://www.ecmwf.int/en/newsletter/169/computing/advanced-regridding-metview).

In [ ]:
import metview as mv

### Módulo climetlab

[Climetlab](https://climetlab.readthedocs.io/en/latest/) es un paquete de Python cuyo objetivo es simplificar el acceso a datos meteorológicos y climáticos, para que los usuarios puedan dedicarse al estudio de los datos y no a resolver problemas técnicos como el acceso y formato de los datos. Está todavía en desarrollo (de hecho puede que sea sustituido por otro paquete llamado earthkit).

In [ ]:
import climetlab as cml

Un ejemplo utilizando datos libres:

In [ ]:
source = cml.load_source("ecmwf-open-data", param=["2t", "msl"],)
for s in source:
    cml.plot_map(s, title=True)
source.to_xarray()

Este código sólo funcionará si se dispone de una API key para acceder al Climate Data Store:

In [ ]:
source = cml.load_source(
    "cds",
    "reanalysis-era5-single-levels",
    variable=["2t", "msl"],
    product_type="reanalysis",
    area=[50, -50, 20, 50],
    date="2011-12-02", # ["2011-12-02", "2011-12-03"] for several dates
    time="12:00",
)
for s in source:
    cml.plot_map(s)